In [ ]:
import os
import numpy as np
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
CHANNEL_NAMES = [
    "Fp1","Fp2","F7","F3","Fz","F4","F8",
    "T3","C3","Cz","C4","T4",
    "T5","P3","Pz","P4","T6",
    "O1","O2"
]

In [3]:
LEFT_CHANNELS = ["Fp1","F7","F3","T3","C3","T5","P3","O1"]
RIGHT_CHANNELS = ["Fp2","F8","F4","T4","C4","T6","P4","O2"]

LEFT_IDX = [CHANNEL_NAMES.index(ch) for ch in LEFT_CHANNELS]
RIGHT_IDX = [CHANNEL_NAMES.index(ch) for ch in RIGHT_CHANNELS]

In [4]:
WINDOW_SIZE = 256
STEP_SIZE = 128

def create_windows(eeg, window_size, step_size):
    windows = []
    for i in range(0, eeg.shape[1] - window_size, step_size):
        windows.append(eeg[:, i:i+window_size])
    return np.array(windows)


In [5]:
def load_folder_hemisphere(folder_path, label, hemisphere_idx):
    X_list = []
    y_list = []

    files = [f for f in os.listdir(folder_path) if f.endswith('.mat')]

    for file in files:
        mat = loadmat(os.path.join(folder_path, file))
        for key in mat.keys():
            if not key.startswith("__"):
                eeg = mat[key]
                break

        eeg = eeg.T
        eeg = eeg[hemisphere_idx, :]  # 👈 HEMISPHERE SELECTION

        windows = create_windows(eeg, WINDOW_SIZE, STEP_SIZE)

        X_list.append(windows)
        y_list.extend([label] * len(windows))

    return np.concatenate(X_list), np.array(y_list)


In [6]:
# LEFT hemisphere
X_L_adhd, y_L_adhd = load_folder_hemisphere("data/ADHD_part1", 0, LEFT_IDX)
X_L_ctrl, y_L_ctrl = load_folder_hemisphere("data/Control_part1", 1, LEFT_IDX)

# RIGHT hemisphere
X_R_adhd, y_R_adhd = load_folder_hemisphere("data/ADHD_part1", 0, RIGHT_IDX)
X_R_ctrl, y_R_ctrl = load_folder_hemisphere("data/Control_part1", 1, RIGHT_IDX)

In [7]:
X_left = np.concatenate([X_L_adhd, X_L_ctrl])
y_left = np.concatenate([y_L_adhd, y_L_ctrl])

X_right = np.concatenate([X_R_adhd, X_R_ctrl])
y_right = np.concatenate([y_R_adhd, y_R_ctrl])

In [ ]:
def build_cnn_hemisphere(input_shape):
    model = Sequential([
        Conv2D(16, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2,2)),

        Conv2D(32, (3,3), activation='relu'),
       
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model



In [22]:
model_left = build_cnn((len(LEFT_IDX),256,1))
model_right = build_cnn((len(RIGHT_IDX),256,1))

In [23]:
from sklearn.model_selection import train_test_split

X_left_train, X_left_test, y_left_train, y_left_test = train_test_split(
    X_left, y_left,
    test_size=0.2,
    random_state=42,
    stratify=y_left
)



In [24]:
X_right_train, X_right_test, y_right_train, y_right_test = train_test_split(
    X_right, y_right,
    test_size=0.2,
    random_state=42,
    stratify=y_right
)


In [14]:
print("X_left_train shape:", X_left_train.shape)
print("X_right_train shape:", X_right_train.shape)



X_left_train shape: (6565, 8, 256)
X_right_train shape: (6565, 8, 256)


In [25]:
# Add channel dimension for LEFT hemisphere
X_left_train = X_left_train[..., np.newaxis]
X_left_test  = X_left_test[..., np.newaxis]

# Add channel dimension for RIGHT hemisphere
X_right_train = X_right_train[..., np.newaxis]
X_right_test  = X_right_test[..., np.newaxis]


In [16]:
print("X_left_train shape:", X_left_train.shape)
print("X_right_train shape:", X_right_train.shape)


X_left_train shape: (6565, 8, 256, 1)
X_right_train shape: (6565, 8, 256, 1)


In [18]:
print("y_left_train shape:", y_left_train.shape)
print("y_left_train dtype:", y_left_train.dtype)
print("Unique labels:", np.unique(y_left_train))



y_left_train shape: (6565,)
y_left_train dtype: int64
Unique labels: [0 1]


In [26]:
# Rebuild models using actual data shape (SAFE METHOD)
model_left = build_cnn_hemisphere(X_left_train.shape[1:])
model_right = build_cnn_hemisphere(X_right_train.shape[1:])


C:\Users\ojesh_hpnmmdy\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
history_left = model_left.fit(
    X_left_train, y_left_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)


Epoch 1/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5779 - loss: 3.8051 - val_accuracy: 0.5933 - val_loss: 0.6855
Epoch 2/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5834 - loss: 0.6895 - val_accuracy: 0.5933 - val_loss: 0.6803
Epoch 3/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5842 - loss: 0.6853 - val_accuracy: 0.5918 - val_loss: 0.6925
Epoch 4/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5834 - loss: 0.6818 - val_accuracy: 0.5918 - val_loss: 0.6783
Epoch 5/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5840 - loss: 0.6837 - val_accuracy: 0.5918 - val_loss: 0.6776
Epoch 6/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5836 - loss: 0.6795 - val_accuracy: 0.5918 - val_loss: 0.6770
Epoch 7/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5836 - loss: 0.6792 - val_accuracy: 0.5918 - val_loss: 0.6767
Epoch 8/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5836 - loss: 0.6792 - val_accuracy: 0.

In [28]:
history_right = model_right.fit(
    X_right_train, y_right_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

Epoch 1/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.5920 - loss: 3.1257 - val_accuracy: 0.7403 - val_loss: 0.6505
Epoch 2/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.6706 - loss: 0.6400 - val_accuracy: 0.7982 - val_loss: 0.6068
Epoch 3/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5889 - loss: 0.6870 - val_accuracy: 0.5933 - val_loss: 0.6797
Epoch 4/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5847 - loss: 0.6777 - val_accuracy: 0.5925 - val_loss: 0.6693
Epoch 5/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5855 - loss: 0.6766 - val_accuracy: 0.5941 - val_loss: 0.6729
Epoch 6/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5843 - loss: 0.6687 - val_accuracy: 0.5941 - val_loss: 0.6747
Epoch 7/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5840 - loss: 0.6807 - val_accuracy: 0.5925 - val_loss: 0.6773
Epoch 8/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5836 - loss: 0.6795 - val_accuracy

In [29]:
right_loss, right_acc = model_right.evaluate(X_right_test, y_right_test)
left_loss, left_acc = model_left.evaluate(X_left_test, y_left_test)

print(f"Left Hemisphere Accuracy : {left_acc:.4f}")
print(f"Right Hemisphere Accuracy: {right_acc:.4f}")


52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5859 - loss: 0.6779
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5853 - loss: 0.6786
Left Hemisphere Accuracy : 0.5853
Right Hemisphere Accuracy: 0.5859
